In [1]:
from __future__ import print_function
# __future__ is a module where we can extract the latest version features, so as we imported print_function here, we can use latest version of python print_function from __future__ module.
import torch
# Import torch module, torch is a open source framework developed by facebook team. It is a python package that has tensor computation with gpu accelration
# deep neural networks build on autograd system
import torch.nn as nn
## nn is a neural network in the torch package, it contains different classes/methods to build neural networks
import torch.nn.functional as F
## nn.Function conventioned as F, is a Function containing all functions such as activation functions, loss functions,convolutional layers
import torch.optim as optim
## Optimizers are parameters that are used to update the weights,bias after backpropogation. torch.optim as optim contains all the optimizers module.
from torchvision import datasets, transforms
## datasets: Abstract interface of objects with a __len__ and __getitem__, including classes as TensorDataset
## transforms: image transformations

In [2]:
## Creating class called Net, by passing nn.Module which has lot of attributes,methods useful for training the neural networks.
class Net(nn.Module):
    ## Instantiating the attributes
    def __init__(self):
        # Inhering properties of Net class.
        super(Net, self).__init__()
        # Starting Convolution process, using nn.Conv2D, with arguments inside paranthesis as (input channels, output channles, kernel size, padding)
        # input channels: channel of input image
        # output channels: channel of output image == number of kernels to be moved at a input image
        # kernel size: size of kernel to extract features.
        # padding: To add to boundary of image pixels, so that no loss of information is lost at pixel level of edge pixels. 
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        # InputChannel: 1, number of kernels: 32, kernelSize: 3, padding: 1 
        # As per network, given input is of 28*28 with 1 channel
        # so, when padding is added, it will be 30*30 
        # Receptive Field will be (28,28,32)

        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # Input: 32, output(kernels to be applied): 64, kernelsize:3, padding: 1
        # Upon padding, (28,28,32) -- > (30,30,32)
        # Applying 64 kernels of size 3 --> (28,28,64)

        self.pool1 = nn.MaxPool2d(2, 2)
        # Applying Maxpoolling of (2*2) matrix,
        # (28,28,64) ---pooling--> (14,14,64)

        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        # Input: 64, output(kernels to be applied): 128, kernelsize:3, padding: 1
        # Upon padding, (14,14,64) -- > (16,16,64)
        # Applying 128 kernels of size 3 --> (14,14,128)

        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        # Input: 128, output(kernels to be applied): 256, kernelsize:3, padding: 1
        # Upon padding, (14,14,128) -- > (16,16,128)
        # Applying 256 kernels of size 3 --> (14,14,256)

        self.pool2 = nn.MaxPool2d(2, 2)
        # Applying Maxpoolling of (2*2) matrix,
        # (14,14,256) ---pooling--> (7,7,256)

        self.conv5 = nn.Conv2d(256, 512, 3)
        # Input: 256, output(kernels to be applied): 512, kernelsize:3
        # Applying 512 kernels of size 3 on (7,7,256) --> (5,5,512)

        self.conv6 = nn.Conv2d(512, 1024, 3)
        # Input: 512, output(kernels to be applied): 1024, kernelsize:3
        # Applying 1024 kernels of size 3 on (5,5,512) --> (3,3,1024)

        self.conv7 = nn.Conv2d(1024, 10, 3)
       # Input: 1024, output(kernels to be applied): 10, kernelsize:3
        # Applying 10 kernels of size 3 on (3,3,1024) --> (1,1,10)
    
    # handles forward propogation
    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        # RElu activation is used which suppreses negative values, for 1st convolution layer 
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        # RElu activation is used which suppreses negative values, for 3rd convolution layer 
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        # RElu activation is used which suppreses negative values, for 5th convolution layer 
        ## x = F.relu(self.conv7(x))
        x = self.conv7(x)
        # RElu activation is used which suppreses negative values, for 7th convolution layer 
        x = x.view(-1, 10)
        # view as an array of size 10 
        return F.log_softmax(x)
        # Returns the maximum number index

    # The network from forward function looks like:
    # conv1() --> ReLU() --> conv2() --> ReLU() --> Pooling -- >conv3() --> ReLU() --> conv4() --> ReLU() --> Pooling
    # --> conv5() --> ReLU() --> Conv6() --> ReLU() --> conv7() --> ReLU() 
    

In [3]:
!pip install torchsummary
# Installing torchsummary module
from torchsummary import summary
# provides information abt the model we designed
use_cuda = torch.cuda.is_available()
print(use_cuda)
# Check if cuda is available, returns True
device = torch.device("cuda" if use_cuda else "cpu")
# If cuda is available , it uses gpu else cpu
model = Net().to(device)
# model is called.
summary(model, input_size=(1, 28, 28))
# Provides summary of model with networks, layers.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
True
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (

<ipython-input-2-a6ad2efba56a>:66: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:


torch.manual_seed(1) # For generating random numbers
batch_size = 128 # set the batchsize as 128, so we only take 128 as a batch.

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
# setting up keys if gpu is available.

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=False, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=False, **kwargs)
## train_loader and test_loader are defined.
# train_loader, gets the dataset and obtain some for training, and also the values are noramlized and converted to tensorform.
# test_loader, gets the testing data from dataset mnist, and normalize the values after converting to tensor form.
# Both have batch size of 128, and are shuffled for every epoch.

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
from tqdm import tqdm
#tqdm is a Python library for adding progress bar.
# It lets you configure and display a progress bar with metrics you want to track.
# Its ease of use and versatility makes it the perfect choice for tracking machine learning experiments

# Training function is defined.
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    # starts
    pbar = tqdm(train_loader)
    # Gets the progress bar
    # enumerate through index and data
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        # update the weights as in backpropogation
        output = model(data)
        loss = F.nll_loss(output, target)
        # Calculates the loss function
        loss.backward()
        # processes backpropogation
        optimizer.step()
        # .step() indicates to move to next parameter
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        # returns the metric

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    # initiate variable
    with torch.no_grad(): # differnation is set to false in this line
        for data, target in test_loader: # iterates through test data
            data, target = data.to(device), target.to(device)
            output = model(data) 
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    # calc. loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:


model = Net().to(device)
# recalls the model
optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9)
# optimzier updates the parameters

# model is iterated for 2 epochs
for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    # thrugh training and testing functions which calc, and updates paramters and finally calc. losses & learns.
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-a6ad2efba56a>:66: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.2742871344089508 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.40it/s]



Test set: Average loss: 0.0580, Accuracy: 9825/10000 (98%)



In [7]:

## If one relu is removed from last layer, accuracy goes as mentioned above.
## Else it will be less than 30%. 